In [19]:
# Enable module reloading
%load_ext autoreload
%autoreload 2

%load_ext tensorboard

In [21]:
import sys
sys.path.append('../..')
from src.models import *

%run -i 'train_MLP.py'

[ 7.094  8.344  2.969 ... -8.25   9.969 -0.031]
[<tf.Tensor 'input_1_11:0' shape=(None, 1) dtype=int16>, <tf.Tensor 'input_2_11:0' shape=(None, 1) dtype=int16>]
Train on 14400 samples, validate on 1600 samples
Epoch 1/2
[<tf.Tensor 'IteratorGetNext:0' shape=(32, 1) dtype=int16>, <tf.Tensor 'IteratorGetNext:1' shape=(32, 1) dtype=int16>]
[<tf.Tensor 'IteratorGetNext:0' shape=(32, 1) dtype=int16>, <tf.Tensor 'IteratorGetNext:1' shape=(32, 1) dtype=int16>]
14400/14400 [==============================] - 3s 213us/sample - loss: 4.2927 - val_loss: 4.0776
Epoch 2/2
14400/14400 [==============================] - 2s 120us/sample - loss: 3.7646 - val_loss: 4.0091


In [18]:
import sys
sys.path.append('../..')
from src.models import *

%run -i 'train_GMF.py'

TypeError: __init__() missing 1 required positional argument: 'n_rows'

# MLP

In [13]:
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
import pandas as pd
from sklearn.metrics import mean_absolute_error

dataset = pd.read_csv('../../data/Jester-Dataset-ratings.csv', delimiter=',')
dataset = dataset[:20000]

# zabezpecenie aby ID boli v rozmedzi (0, pocet_userov/jokov)
dataset['USER_ID'] = dataset['USER_ID'].astype('category').cat.codes.values
dataset['JOKE_ID'] = dataset['JOKE_ID'].astype('category').cat.codes.values


# rozdelenie datasetu
train, test = train_test_split(dataset, test_size=0.2)
y_true = test['Rating']

number_users = len(dataset['USER_ID'].unique())
number_jokes = len(dataset['JOKE_ID'].unique())


# Debug
print(train.shape)
print(test.shape)
print(y_true.shape)
print(number_users)
print(number_jokes)

# Nahodne vymyslene cislo, predstavuju shape vystupu z embedding vrstiev

emb_output_dim = 5 


# embeddingy sa vytvoria zvlast pre joke a usera a potom sa concatnu

joke_input = keras.layers.Input(shape=[1])
joke_emb = keras.layers.Embedding(number_jokes + 1, emb_output_dim)(joke_input)
joke_vector = keras.layers.Flatten()(joke_emb)

user_input = keras.layers.Input(shape=[1])
user_emb = keras.layers.Embedding(number_users + 1, emb_output_dim)(user_input)
user_vector = keras.layers.Flatten()(user_emb)

concat = keras.layers.concatenate([joke_vector, user_vector])

# skryte vsrtvy
dense = keras.layers.Dense(units=32, activation='relu')(concat)
dense_2 = keras.layers.Dense(units=16, activation='relu')(dense)
dense_3 = keras.layers.Dense(units=12, activation='relu')(dense_2)
output = keras.layers.Dense(units=1, activation='relu')(dense_3)


model = keras.Model([user_input, joke_input], output)

model.compile(
    optimizer='adam',
    loss= 'mean_absolute_error'
)

model.fit(
    [np.array(train['USER_ID']), np.array(train['JOKE_ID'])], 
    np.array(train['Rating']), 
    epochs=10, 
    verbose=1, 
    validation_split=0.1
)


# Vyhodnotenie

print(model.summary())

from sklearn.metrics import mean_absolute_error
y_pred = model.predict([np.array(test['USER_ID']), np.array(test['JOKE_ID'])])


tmp = pd.DataFrame()
tmp['user_id'] = test['USER_ID']
tmp['joke_id'] = test['JOKE_ID']
tmp['predicted_rating'] = y_pred
tmp['true_rating'] = y_true
tmp

(16000, 3)
(4000, 3)
(4000,)
466
140
Train on 14400 samples, validate on 1600 samples
Epoch 1/10
14400/14400 [==============================] - 3s 225us/sample - loss: 4.3462 - val_loss: 3.8146
Epoch 2/10
14400/14400 [==============================] - 2s 127us/sample - loss: 3.8106 - val_loss: 3.7793
Epoch 3/10
14400/14400 [==============================] - 2s 131us/sample - loss: 3.7658 - val_loss: 3.7410
Epoch 4/10
14400/14400 [==============================] - 2s 129us/sample - loss: 3.7416 - val_loss: 3.7390
Epoch 5/10
14400/14400 [==============================] - 2s 136us/sample - loss: 3.7227 - val_loss: 3.7322
Epoch 6/10
14400/14400 [==============================] - 2s 135us/sample - loss: 3.7038 - val_loss: 3.7353
Epoch 7/10
14400/14400 [==============================] - 2s 133us/sample - loss: 3.6839 - val_loss: 3.7224
Epoch 8/10
14400/14400 [==============================] - 2s 138us/sample - loss: 3.6594 - val_loss: 3.7088
Epoch 9/10
14400/14400 [==========================

KeyboardInterrupt: 

In [5]:
print(mean_absolute_error(y_true, y_pred))

3.8974805486772954
